In [1]:
print("Hello World")

Hello World


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("pyspark-notebook").master("spark://spark-master:7077").config("spark.executor.memory", "512m").getOrCreate()

#dataframe = spark.read.format("jdbc").options(url="jdbc:postgresql://db:5432/datascience?user=postgres&password=mantequilla", dbtable="rutas",driver="org.postgresql.Driver").load()
buses = spark.read.option("header",True).csv("./data/buses.csv")
print("Hola")

Hola


In [9]:
paraderos2 = spark.read.option("header",True).csv("./data/paraderos2.csv")
paraderosrutas = spark.read.option("header",True).csv("./data/paraderosrutas.csv")

In [28]:
# Observacion: Hay que cambiar la obtencion de parder_geom, puesto que debiese venir de la tabla paraderos2 pero el csv de stops.csv venía sin ese campo
temp = paraderosrutas.join(buses, buses.vendor == paraderosrutas.cod_sinser).select(paraderosrutas.simt, 
    buses.bus, buses.fechahora, buses.lat, buses.lng, buses.velocidad, buses.pasajeros, buses.geom, buses.vendor, buses.id,
    paraderosrutas.servicio, paraderosrutas.nombre_par, paraderosrutas.geom.alias("parder_geom"))

print(paraderos2.columns)
busesparaderos = temp.join(paraderos2, paraderos2.stop_id == temp.simt, 'inner').select(temp.columns + [paraderos2.stop_id.alias('parader'), 
    paraderos2.stop_name, paraderos2.stop_lat.alias('parder_lat'), paraderos2.stop_lon.alias('parder_lon')
])
print(busesparaderos.columns)
print(temp.columns)

['stop_id', 'stop_code', 'stop_name', 'stop_lat', 'stop_lon', 'stop_url']
['simt', 'bus', 'fechahora', 'lat', 'lng', 'velocidad', 'pasajeros', 'geom', 'vendor', 'id', 'servicio', 'nombre_par', 'parder_geom', 'parader', 'stop_name', 'parder_lat', 'parder_lon']
['simt', 'bus', 'fechahora', 'lat', 'lng', 'velocidad', 'pasajeros', 'geom', 'vendor', 'id', 'servicio', 'nombre_par', 'parder_geom']


In [34]:
print(buses.schema)
#minimos = busesparaderos.groupBy('fechahora').agg(F.min(busesparaderos.d).alias("md")).select(busesparaderos.distance, busesparaderos.fechahora.alias("fh"))
#busesparaderos = dataframe.join(minimos, dataframe.d == minimos.md & dataframe.fechahora == minimos.fh, 'inner')

StructType(List(StructField(bus,StringType,true),StructField(fechahora,StringType,true),StructField(lat,StringType,true),StructField(lng,StringType,true),StructField(velocidad,StringType,true),StructField(pasajeros,StringType,true),StructField(geom,StringType,true),StructField(vendor,StringType,true),StructField(id,StringType,true)))


In [ ]:
def get_Bus_Cant_Prom_Por_Paredero():#Frecuenica
    Hour_list_from = ['2018-06-13 00:00:00+00','2018-06-13 01:00:00+00','2018-06-13 02:00:00+00','2018-06-13 03:00:00+00','2018-06-13 04:00:00+00','2018-06-13 05:00:00+00','2018-06-13 06:00:00+00','2018-06-13 07:00:00+00','2018-06-13 08:00:00+00','2018-06-13 09:00:00+00','2018-06-13 10:00:00+00','2018-06-13 11:00:00+00','2018-06-13 12:00:00+00','2018-06-13 13:00:00+00','2018-06-13 14:00:00+00','2018-06-13 15:00:00+00','2018-06-13 16:00:00+00','2018-06-13 18:00:00+00','2018-06-13 19:00:00+00','2018-06-13 20:00:00+00','2018-06-13 21:00:00+00','2018-06-13 22:00:00+00','2018-06-13 23:00:00+00']

    Hour_list_to = ['2018-06-13 00:59:59+00','2018-06-13 01:59:59+00','2018-06-13 02:59:59+00','2018-06-13 03:59:59+00','2018-06-13 04:59:59+00','2018-06-13 05:59:59+00','2018-06-13 06:59:59+00','2018-06-13 07:59:59+00','2018-06-13 08:59:59+00','2018-06-13 09:59:59+00','2018-06-13 10:59:59+00','2018-06-13 11:59:59+00','2018-06-13 12:59:59+00','2018-06-13 13:59:59+00','2018-06-13 14:59:59+00','2018-06-13 15:59:59+00','2018-06-13 16:59:59+00','2018-06-13 18:59:59+00','2018-06-13 19:59:59+00','2018-06-13 20:59:59+00','2018-06-13 21:59:59+00','2018-06-13 22:59:59+00','2018-06-13 23:59:59+00']
    #*************************
    #print(type(result.index))
    #print(result.index.names)
    #print(result.index.values)
    #print(result.index.levels)

    dataframe_ruta = pd.read_csv("./E04I.csv")
    temp2 = dataframe_ruta.groupby(['parader'], group_keys=True, as_index=True).apply(lambda x: x)
    #temp4 = dataframe_ruta.groupby(['parder_geom'], group_keys=True, as_index=True).apply(lambda x: x)
    list_paredors = temp2.index.levels[0]
    #temp3 = temp2.loc['PA146']
    #a = temp2.loc['PA146']['fechahora'].tolist()[0]

    dataframe_buses = {'paradero':[],'promedio_bus_hora':[],'geo_parader':[],'geo_parader_lat':[],'geo_parader_lon':[],'calificacion':[]}
    #print(len(Hour_list_from))
    #print(len(Hour_list_to))

    for i in list_paredors: 
        x = temp2.loc[i]
        List_total_buses = list()
        geo = x['parder_geom'].iat[0]
        geo_lat = x['parder_lat'].iat[0]
        geo_lon = x['parder_lon'].iat[0]
        paradero_list = list()
        #print(geo)
        for j in range(23):
            #print(j)
            df = x[(x['fechahora'] >= Hour_list_from[j]) & (x['fechahora'] <= Hour_list_to[j])]
            List_total_buses.append(df['bus'].nunique())
            #Lista_interval.append(Hour_list_from[j]+' to '+Hour_list_to[j])
        mean = (sum(List_total_buses) / len(List_total_buses))
        dataframe_buses['paradero'].append(i)
        dataframe_buses['promedio_bus_hora'].append(mean)
        dataframe_buses['geo_parader'].append(geo)
        dataframe_buses['geo_parader_lat'].append(geo_lat)
        dataframe_buses['geo_parader_lon'].append(geo_lon)
        if mean < 5:
            dataframe_buses['calificacion'].append(0)
        elif mean > 6:
            dataframe_buses['calificacion'].append(1)
        else:
            dataframe_buses['calificacion'].append(2)


        
        #dataframe_buses['rango_hora'] += Lista_interval


    dataframe_total_busparadero = pd.DataFrame(dataframe_buses)
    print(dataframe_total_busparadero)
    dataframe_total_busparadero.to_csv('E04I_NOTED.csv')